<a href="https://colab.research.google.com/github/kslim1025/TF_Keras_RNN/blob/master/ImageNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# MNIST 문자열 분석으로 유명한 데이터셋이 패션에 관한 데이터 세트를 만듬 그게 FashionMINIST 데이터셋이다.
# 데이터 이미지가 0에서 255까지 값을 가지는 28x28이미지라는 것을 확인가능
# 정답이 되는 라벨을 확인하기 위해 print를 붙여서  확인
# 외부 데이터를 이용한 정제과정
# ctrl+enter를 사용한 런타임가능
# !nvidia-smi : 어떤 GPU를 사용하는지 확인가능한 명령어 
#

try:
  # %tensorflow_version only exists in Colab
  %tensorflow_version 2.x
except Exception:
  pass

# 넘파이는 수학과 과학 연산에 특화된 파이썬 모듈로 딥러닝에서도 유용하게 사용된다.

import numpy as np;
import tensorflow as tf;
import pandas as pd;
import matplotlib.pyplot as plt;
import math;
import re;
################################################################################
!nvidia-smi
       
#+-----------------------------------------------------------------------------+
#| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
#|-------------------------------+----------------------+----------------------+
#| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
#| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
#|                               |                      |               MIG M. |
#|===============================+======================+======================|
#|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
#| N/A   51C    P0    35W / 250W |   1581MiB / 16280MiB |      0%      Default |
#|                               |                      |                 ERR! |
#+-------------------------------+----------------------+----------------------+
#                                                                               
#+-----------------------------------------------------------------------------+
#| Processes:                                                                  |
#|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
#|        ID   ID                                                   Usage      |
#|=============================================================================|
#|  No running processes found                                                 |
#+-----------------------------------------------------------------------------+

################################################################################

# 텐서플로 2.0을 이용한 턴서플로 허브 불러오는 방법

#1. 텐서플로 허브에서 사전 훈련된 MobileNet 모델 불러오기
import tensorflow_hub as hub

mobile_net_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2"
model = tf.keras.Sequential([
  hub.KerasLayer(handle = mobile_net_url, input_shape=(224,224,3), trainable=False)
])

model.summary()

# MobileNet 이란?

#MobileNet은 계산 부담이 큰 컨볼루션 신경망을 연산 성능이 제한적인 모바일 환경에서도 작동 가능하도록 네트워크 구조를 경량화한것입니다.
#MobileNet 버전2는 1을 개선했고 파라미터 수도 더 줄어들었습니다.

#MobileNet은 ImageNet에 존재하는 1,000 종류의 이미지를 분류할 수 있으며, 이 가운데 어떤것에도 속하지 않는다고 판단될 때는 background에 해당 하는 인덱스 0을 반환합니다. 이미지의 분류는 수량(cock)과 암탉(hen)을 분류할 정도로 상세하고 화장지(toilet tissue)같은 사물도 포함되어있다.

#MobileNet의 성능을 평가하기 위해 이미지를 학습시켰을 때 얼마나 적합한 라벨로 분류하는지 알아보겠습니다.
#ImageNet의 데이터 중 일부만 모아놓은 ImageNetV2를 사용하겠습니다. ImageNetV2는 아마존 매커니컬 터크를 이용해 다수의 참가자에게서 클래스 예측값을 받아서 선별한 데이터입니다. 여기서는 각클래스에서 가장 많은 선택을 받은 이미지 10장씩 모아놓은 10,000장의 이미지가 포함된 TopImages 데이터를 사용하겠습니다.

# imageNetV2-TopImages 불러오기

import os
import pathlib

content_data_url = '/content/sample_data'
data_root_orig = tf.keras.utils.get_file('imagenetV2', 'https://s3-us-west-2.amazonaws.com/imagenetv2public/imagenetv2-topimages.tar.gz', cache_dir=content_data_url, extract=True)
data_root= pathlib.Path(content_data_url + '/datasets/imagenetv2-topimages')
print(data_root)

# 디렉터리 출력
#for idx, item in enumerate(data_root.iterdir()):
for idx, item in enumerate('C:/Users/kslim/Desktop/datasets/imagenetv2-topimages'):
  print(item)
  if idx == 9:
    break

# ImageNet 라벨 텍스트 불러오기

label_file = tf.keras.utils.get_file('label', 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')

label_text = None
with open(label_file, 'r') as f:
  label_text = f.read().split('\n')[:-1]
print(len(label_Text))
print(label_text[:10])
print(label_text[-10:])

# 이미지 확인

import PIL.Image as Image
import matplotlib.pyplot as plt
import random

all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]

#이미지를 랜덤하게 섞습니다
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
print('image_count', image_count)

plt.figure(figsize=(12, 12))

for c in range(9):
  image_path = random.chice(all_image_paths)
  plt.subplot(3,3,c+1)
  plt.imshow(plt.imread(image_path))
  idx = int(image_path.split('/')[-2]) + 1
  plt.title(str(idx) + ', ' + label_text[idx])
  plt.axis('off')
plt.show()

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_8 (KerasLayer)   (None, 1001)              3540265   
Total params: 3,540,265
Trainable params: 0
Non-trainable params: 3,540,265
_________________________________________________________________
C
:
/
U
s
e
r
s
/
k
image_count 0


AttributeError: ignored

<Figure size 864x864 with 0 Axes>